# Day 4 - calculation of statistical errors

### Import libraries and data (potential energies)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Run following cells in order to load the energies

In [3]:
!git clone https://github.com/IvanGilardoni/Exercise_error_analysis.git

Clone in 'Exercise_error_analysis' in corso...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 47 (delta 0), reused 47 (delta 0), pack-reused 0
Ricezione degli oggetti: 100% (47/47), 6.32 MiB | 9.29 MiB/s, fatto.


In [5]:
ene = {}
Ts = np.around(np.linspace(0.1, 3, 30), decimals=1)

for temp in Ts:
    ene[temp] = np.loadtxt('Exercise_error_analysis/Energies/potential_energy_%.1f' % temp)

discard equilibration steps and make stride = 10

In [7]:
for temp in Ts:
    ene[temp] = ene[temp][1000:][::10]

## Point 0 - autocorrelation

The trajectories generated in Molecular Dynamics simulations are affected by time correlations: consecutive frames are not independent from each other, rather they depend on the previous ones, up to a certain time difference, after which the correlation effects become negligible. A good quantity to measure this is the **autocorrelation function**, defined as

\begin{equation}
C(\Delta t) = \frac{1}{N}\sum_{i=1}^N \Bigl(E(t_i)-\langle E\rangle\Bigr)\Bigl(E(t_i+\Delta t)-\langle E \rangle\Bigr)
\end{equation}


where $E(t)$ is the time series we are analysing (in our case, the energy) and $\langle E \rangle$ its average value.

Compute the autocorrelation of the energies at different temperatures. What happens for long time differences $\Delta t$? Do we have the same behaviour for all the temperatures?

Compare with uncorrelated white noise: what is the main difference?

## Point 1 - block analysis

**Block analysis** (Flyvbjerg, Petersen; JCP 1989) is a method used to correctly estimate the error on the mean in the case of correlated time series.

1. Recompute the average potential energies (column 3) as done in day 2 (for different temperatures from 0 to 3), together with the corresponding error on the mean estimated by the standard deviation of the mean $\sigma_{\overline{x}}$ (equal to standard deviation divided by square root of n. of samples) . Remember to discard initial steps due to equilibration.

2. Then, write an algorithm to perform block analysis:
- a. focus on the time series of the energy at a given temperature (e.g., 3.0);
- b. split the time series in blocks of a given size;
- c. for each block, compute the average energy;
- d. compute the standard deviation of the mean on the obtained average energies, this is your estimate of the error for the given block size;
- e. repeat b,c,d items for different block size (take values from 1 to half of the trajectory length);
- f. plot the error as a function of the block size. Search for a plateau region, which identifies the optimal block size.


3. Then, use your algorithm with this optimal block size for all the temperatures (assumption). Plot both $\sigma_{\overline{x}}$ and the error estimated with block analysis as a function of the temperature. Do the two estimates agree with each other? 

4. We assumed that the optimal block size is approximatively the same for all the temperatures. Do you expect block analysis to work also at the transition temperature $T=0.6$? Repeat block analysis at that temperature with different block size and search for a plateau.

5. Repeat block analysis for random white noise and plot the result (error vs. block size). Does it agree with the error on the mean estimated by the standard deviation $\sigma_{\overline{x}}$?

6. In this point we are going to estimate the error on the average heat capacity through block analysis. You just have to modify your algorithm (item 2) at point c. by computing the heat capacity on each block rather than the average energy. Then, repeat what previously done, namely:
- focus on a given temperature (e.g., 3.0) and identify the optimal block size searching for a plateau;
- use this block size for all the temperatures and compute the error on the average heat capacity; plot it as a function of the temperature;
- focus on the transition temperature: does block analysis work in this case?

## Point 2 - bootstrap on blocks

**Bootstrap** (Efron, 1979) is a procedure to compute properties of an estimator by random re-sampling with replacement from the data. Write an algorithm to perform boostrap on blocks:
1. split your time series into $N$ (consecutive and disjoint) blocks, as done in block analysis (use the optimal $N$ you have got in the previous point);
2. generate a dummy trajectory by concatenating $N$ randomly sampled blocks with replacement (so that the trajectory has the same length as the original one) and use it to compute your quantity of interest $O_i$ (such as the mean energy or the heat capacity) (note: differently from block analysis, we are now computing our quantity using every data point on the newly sampled trajectory);
3. repeat 2nd step several times (e.g. $i=1...100$), then take the average and standard deviation of the $O_i$ values; the standard deviation will estimate the uncertainty on the average value.

Use your algorithm to compute the error on the mean of the potential energy, as done in the previous point. Does this estimate agree with the previous one? Plot the two estimates as a function of the temperature.

Now, use your algorithm to compute the statistical error on the heat capacity, computed from fluctuations of the potential energy. Compare with the calculation of the heat capacity given by the derivative of the energy with respect to the temperature, approximated through finite difference.

What happens if you neglected to remove initial equilibration steps? 

Finally, compute the values of heat capacity and the associated errors with the trajectories generated starting from the liquid phase (equilibrium configuration at $T=3$). Compare with previous values: where do you observe significative discrepancies? For such temperatures, did you correctly estimate the statistical error on the specific heat?

In [ ]:
ene_back = {}

Ts_back = list(np.around(np.linspace(0.1, 1, 10), decimals=1)) + [1.5, 2.0, 2.5, 3.0]
Ts_back = np.array(Ts_back)

for temp in Ts_back:
    ene_back[temp] = np.loadtxt('Exercise_error_analysis/Energies/potential_energy_back_%.1f' % temp)[1000:][::10]